##1.Train and evaluate reference network: ssd resnet50, batch 4, momentum 0.6

We'll keep the pipeline_new.config file as it was created, no changes in the hyperparameters in order to define the baseline.

##Analysis 
In previous tries we noticed that the model couldn't converge even on the Train set. 
In order to fix it, we lowered the optimizer momentum to 0.6.

The results were clearly overfitting on the trainset. But this was a good baseline to check our model for further improvements (e.g. data augmentation, more training steps, learning rate tuning)


see:
https://medium.com/swlh/tensorflow-2-object-detection-api-with-google-colab-b2af171e81cc


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

##Import libraries

In [ ]:
import sys
sys.path.append('/content/gdrive/MyDrive/GoogleColab/lib')
sys.path.append('/content/gdrive/MyDrive/GoogleColab/lib/models/research/')
sys.path.append('/content/gdrive/MyDrive/GoogleColab/lib/models/research/slim/')
sys.path.append('/content/gdrive/MyDrive/GoogleColab/lib/models/')
sys.path.append('/content/gdrive/MyDrive/GoogleColab/lib/object_detection/')
libs_path = '/content/gdrive/MyDrive/GoogleColab/lib'
%cd $libs_path

##Install tf object detection api

In [ ]:
%cd $libs_path
import os
if 'models' in os.listdir(libs_path):
  %rm -r models
!git clone --depth 1 https://github.com/tensorflow/models


## Substitute trainer.py and model_lib_v2.py with my versions


**trainer.py line 376**
- max num checkpoints: None

**model_lib_v2.py line 1136**
- iterate over checkpoints for evaluation


see: 
https://thachngoctran.medium.com/make-tensorflows-object-detection-validation-a-true-post-process-c45785f08d3a


**model_lib_v2.py line 633**
- normalize images for visualization


##FIND THE MODIFIED SCRIPTS AT THIS LINK FROM MY DRIVE:

https://drive.google.com/file/d/1D6fRZdFjqnM8z3gSxaR7ZrxYcDl_5JW3/view?usp=sharing
https://drive.google.com/file/d/1FnhgfyEQEMEGDAEzAjSF9GsiUzdqJMZ8/view?usp=sharing

you should download them and upload them in your GoogleColab folder. Otherwise, please update your paths in order to make this work.

#####################################################

In [ ]:
import shutil
path_my_files = '/content/gdrive/MyDrive/GoogleColab/'


path_new_model_lib_v2 = path_my_files + 'model_lib_v2.py'
path_new_trainer = path_my_files + 'trainer.py'

path_original_model_lib_v2 = '/content/gdrive/MyDrive/GoogleColab/lib/models/research/object_detection/model_lib_v2.py'
path_original_trainer = '/content/gdrive/MyDrive/GoogleColab/lib/models/research/object_detection/legacy/trainer.py'


#move the new version in
shutil.copyfile(path_new_model_lib_v2, path_original_model_lib_v2)
shutil.copyfile(path_new_trainer, path_original_trainer)

In [ ]:
%cd $libs_path/models/research/
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

In [ ]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q pycocotools

In [ ]:
libs_path = '/content/gdrive/MyDrive/GoogleColab/lib'
!pip3 uninstall opencv-python-headless==4.5.5.62
!pip3 install --target=$libs_path opencv-python-headless==4.5.2.52 


##Navigate to project folder - set my current paths



In [ ]:
path = '/content/gdrive/MyDrive/GoogleColab'
udacity_path = path + '/UDACITY'
sdc_path = udacity_path + '/SDC'
project_path = sdc_path + '/1-waymo-object-detection'


#Google drive
data_path = project_path + '/data'
waymo_path = data_path + '/waymo'
train_dir = data_path + '/train/'
val_dir = data_path + '/val/'
test_dir = data_path + '/test'
experiments_data = data_path + '/experiments_data'
model_ckpts = experiments_data + '/pretrained_model/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0'
tfrecord_test = test_dir + 'segment-12200383401366682847_2552_140_2572_140_with_camera_labels.tfrecord'

current_dir = experiments_data + '/reference2'
pipeline_config_path = current_dir + '/pipeline_new.config'
exported_dir = current_dir + '/exported'
saved_model_dir = exported_dir + '/saved_model'
animation_path = current_dir + '/animation.mp4'

#Github clone
folder_path = project_path + '/Udacity-waymo-object-detection'
workspace_path = folder_path + '/home/workspace'
experiments_path = workspace_path + 'experiments/'
label_map = workspace_path + 'experiments/label_map.pbtxt'



In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
#test if the imports are correct
%cd $workspace_path
!git pull
!python utils_colab.py

##Create pipeline_new.config (do it once)

In [ ]:
'''
#create pipeline_new.config 
%cd $workspace_path
!python edit_config.py --train_dir $train_dir --eval_dir $val_dir --batch_size 2 --checkpoint $model_ckpts --label_map $label_map
'''

##Start Tensorboard

see https://medium.com/swlh/tensorflow-2-object-detection-api-with-google-colab-b2af171e81cc

In [ ]:
#start Tensorboard
%load_ext tensorboard
%tensorboard --logdir=$current_dir 


![](https://drive.google.com/uc?id=1X8CvEeQkazFfWxucWQiqmQ1jOQ7c0Mhe)

![](https://drive.google.com/uc?id=1W6sNtM-YIn1jOXyaYzx8pnfn3uN_V61b)

![](https://drive.google.com/uc?id=18cNSbDgUh6cAaxuAOgrZVd2yXH1QUDvy)

![](https://drive.google.com/uc?id=1jEN_HVfkokyV9M5GBdj6V5IUWEmeEug8)

##Set GPU utils


In [ ]:
print("TF GPU: ",tf.test.gpu_device_name())
!nvcc --version
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0,2,3,4

In [ ]:
%env CUDA_VISIBLE_DEVICES

In [ ]:
%env CUDA_DEVICE_ORDER

##Paths:

```sh
path = '/content/gdrive/MyDrive/GoogleColab'
udacity_path = path + '/UDACITY'
sdc_path = udacity_path + '/SDC'
project_path = sdc_path + '/1-waymo-object-detection'


#Google drive
data_path = project_path + '/data'
waymo_path = data_path + '/waymo'
train_dir = data_path + '/train/'
val_dir = data_path + '/val/'
test_dir = data_path + '/test'
experiments_data = data_path + '/experiments_data'
model_ckpts = experiments_data + '/pretrained_model/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0'
tfrecord_test = test_dir + 'segment-12200383401366682847_2552_140_2572_140_with_camera_labels.tfrecord'

current_dir = experiments_data + '/reference2'
pipeline_config_path = current_dir + '/pipeline_new.config'
exported_dir = current_dir + '/exported'
saved_model_dir = exported_dir + '/saved_model'
animation_path = current_dir + '/animation.mp4'

#Github clone
folder_path = project_path + '/Udacity-waymo-object-detection'
workspace_path = folder_path + '/home/workspace'
experiments_path = workspace_path + 'experiments/'
label_map = workspace_path + 'experiments/label_map.pbtxt'
```

In [ ]:
%workspace_path

##Train reference model

In [ ]:
!python model_main_tf2.py --model_dir=$current_dir --pipeline_config_path=$pipeline_config_path

##Validation

In [ ]:
!python model_main_tf2.py --model_dir=$current_dir --pipeline_config_path=$pipeline_config_path --checkpoint_dir=$current_dir

# Export model

export the model

In [ ]:
#create exported directory
!mkdir $output_directory

!python exporter_main_v2.py --input_type image_tensor --pipeline_config_path $pipeline_config_path --trained_checkpoint_dir $current_dir --output_directory $exported_dir

## Exporting animation

we create a video on the test set to assess performances

In [ ]:
%cd ..

In [ ]:
!python inference_video.py --labelmap_path $label_map --model_path $saved_model_dir  --tf_record_path $tfrecord_test --config_path $pipeline_config_path --output_path $animation_path